# Medidas de aceleração 

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#from math import sqrt

import glob
import os

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

#from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import joblib

# Pré-processamento

Antes de alimentar o modelo, devemos dividir o sinal em janelas (por exemplo, blocos de 2 segundos).

Em cada janela, calcular atributos simples de ax, ay e az. Os atributos são:

1) Média  
 
2) Desvio padrão  

3) Amplitude (máx – mín)
 
4) Média da magnitude 

In [4]:
# O código é de uma função que recebe o DataFrame gerado pelo arquivo CSV e o transforma em outro DataFrame. 
# O novo DataFrame é uma tabela de dados que contém atributos (média, desvio-padrão, amplitude, etc..).
# Os atributos são obtidos a partir do recorte dos dados de aceleração, considerando uma janela de dois segundos. 

def make_windows(df, category='unknown'):
    '''
    A partir de um DataFrame, calculamos, a cada dois segundos, diversos atributos (média, desvio-padrão, amplitude, etc..).
    A saída é outro DataFrame com esses atributos.
    
    df: pandas dataframe
    category: string
    '''

    dt = df.time.max()/(len(df) - 1)
    window_size = 2   # tamanho da janela em segundos
    samples_per_window = int(2/dt)
    
    features = []  # lista para guardar os atributos  
    
    for start in range(0, len(df), samples_per_window):
        window = df.iloc[start:start+samples_per_window]
        if len(window) < samples_per_window:
            continue  # ignora se a última janela ficar incompleta
    
            # Calcula atributos simples
        mean_x = window['ax'].mean()
        std_x  = window['ax'].std()
        amp_x  = window['ax'].max() - window['ax'].min()
        mean_y = window['ay'].mean()
        std_y  = window['ay'].std()
        amp_y  = window['ay'].max() - window['ay'].min()
        mean_z = window['az'].mean()
        std_z  = window['az'].std()
        amp_z  = window['az'].max() - window['az'].min()
        m_abs  = window['abs'].mean()  # média da aceleração absoluta
    
        features.append({
            'mean_x': mean_x, 'std_x': std_x, 'amp_x': amp_x,
            'mean_y': mean_y, 'std_y': std_y, 'amp_y': amp_y,
            'mean_z': mean_z, 'std_z': std_z, 'amp_z': amp_z,
            'm_abs' : m_abs, 
            'classe': category  
        })
    
    # Retornamos os atributos na forma de um dicionário
    return pd.DataFrame(features)

In [5]:
custom_names = ['time', 'ax', 'ay', 'az', 'abs']
 
old_df = pd.DataFrame( )  # Criamos um DataFrame vazio

folder_path = "/home/george/code/accel/data"
files = glob.glob(os.path.join(folder_path, '*.csv'))
for file_path in files:
    df = pd.read_csv(file_path, header=0, names=custom_names)  # Ele gera um DF a partir do arquivo
    category = file_path.split('/')[-1].split('_')[0]   # Utiliza somente o nome do arquivo até o character '_'
    new_df = pd.concat( [old_df, make_windows(df, category)], axis = 0, ignore_index=True)  # Aqui fazemos a concatenação vertical dos dados
    old_df = new_df.copy()
    del df   # Para liberar memória. Se necessário, use a função gc.collect()
    

del old_df
new_df.sample(5)

,mean_x,std_x,amp_x,mean_y,std_y,amp_y,mean_z,std_z,amp_z,m_abs,classe
12,0.015762,0.627805,3.048891,0.157035,0.811628,2.871866,0.249710,0.883378,4.843324,1.248291,walk
25,0.070649,4.099887,17.242791,1.131592,4.071747,15.188600,0.402603,9.575796,42.663078,10.020064,dance
35,0.012189,0.249211,1.087596,0.111247,0.122318,0.581458,0.264241,0.123378,0.528471,0.387081,steady
23,-0.174796,0.597535,2.942857,0.020373,0.905103,2.823171,0.310533,1.080577,5.889541,1.452137,walk
30,-0.011228,0.346852,1.905996,0.021570,0.401861,1.955018,0.286914,0.479043,2.948453,0.653953,steady


Necessitamos de mais dados

# Separar dados para treinamento e teste

Vamos separar os dados em 80% para treinamento e 20% para teste. 

É importante extratificar os dados para evitar desbalanceamento. 

Em seguida, vamos instanciar o modelo de classificação KNN



In [6]:
X = new_df.drop('classe', axis = 1)
y = new_df['classe']

# Separamos 80% dos dados para treinamento e 20% para avaliar o treinamento. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Vamos instanciar o modelo, isto é, vamos especificar o tipo de modelo que será utilizado para o treinamento
knn = KNeighborsClassifier(n_neighbors=3)

# Vamos treinar o modelo com os 80% dos dados já previamente separados
knn.fit(X_train, y_train)

# Vamos avaliar o modelo com os 20% dos dados que não foram usados no treinamento
#  
y_pred = knn.predict(X_test)

# Step 6: Evaluate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))

Accuracy: 1.0
Precision: 1.0
Recall: 1.0


# Testando com um dado que ainda não foi utilizado no treinamento nem na avaliação 

Queremos saber se o modelo realmente funciona.

In [8]:
custom_names = ['time', 'ax', 'ay', 'az', 'abs']
df = pd.read_csv("/home/george/code/accel/teste/dance_george3.csv", header=0, names=custom_names) 
df_desconhecido = make_windows(df) # Em princío não informamos a classe de movimento
X_new = df_desconhecido.drop('classe', axis = 1)

prediction = knn.predict(X_new)
print("Prediction for is:", prediction)

Prediction for is: ['dance' 'dance' 'dance' 'dance' 'dance' 'dance']


👉 Vemos que o modelo foi muito bem treinado. 

No entanto, os dados foram gerados com os movimentos de apenas uma pessoa. Embora sejam movimentos distintos, como andar, ficar parado e dançar, o modelo pode ter tido um super ajuste de acordo com as características específicas da pessoa que gerou os dados. É necessário saber o que acontece com o treinamento do modelo se os dados forem gerados a partir de movimentos de pessoas distintas.  

# Criando um pipeline e preparando o modelo para o App

Vamos usar pipeline. Este pipeline é bem simples.   

In [9]:
pipeline = Pipeline([     
                ('std_scaler', StandardScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=3))
            ] )


In [10]:
pipeline.fit(X_train, y_train)

,steps,"[('std_scaler', ...), ('knn', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,n_neighbors,3
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30


In [11]:
y_predict = pipeline.predict(X_test)

In [12]:

precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)

Precision: 1.0
Recall: 1.0


In [13]:
prediction = pipeline.predict(X_new)

In [14]:
prediction

array(['dance', 'dance', 'dance', 'dance', 'dance', 'dance'], dtype=object)

# Vamos salvar o modelo para criar o app

In [15]:
# Save model to a file
joblib.dump(knn, 'knn_model.joblib')

['knn_model.joblib']

In [16]:
data = {
    "mean_x": 0.1,
    "std_x": 0.6,
    "amp_x": 2.0,
    "mean_y": 0.0,
    "std_y": 0.3,
    "amp_y": 2.0,
    "mean_z": 0.2,
    "std_z": 0.5,
    "amp_z": 4,
    "m_abs": 1.0
}

In [17]:
df_features = pd.DataFrame([data])

In [19]:
pipeline.predict(df_features)[0]

'steady'